In [1]:
import pandas as pd
import numpy as np
import ast


In [2]:
df_movies = pd.read_csv(r'D:\Denise_Estudio\henry\PI\Data\movies_dataset.csv')

C:\Users\ezequ\AppData\Local\Temp\ipykernel_14192\3534112222.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv(r'D:\Denise_Estudio\henry\PI\Data\movies_dataset.csv')


In [3]:
df_movies.shape

(45466, 24)

In [4]:
#funcion extrae datos anidados
def extraer(list):
    try:
        if list != '[]':
            list_aux= ast.literal_eval(list)

            return ', '.join([dic['name'] for dic in list_aux])
        else: 
            return 'no register'
        
    except:
        return 'no register'
    

#funcion extrae datos anidados de cast
def extraer2(list):
    try:
        if list != []:
            list_aux= ast.literal_eval(list)

            return ', '.join([dic['name'] for dic in list_aux])
        else: 
            return 'no register'
        
    except:
        return 'no register'

In [5]:
#extraigo datos anidados en columnas solo guardando 'name' dato de interes
df_movies['genres_clean'] = df_movies['genres'].apply(extraer)
df_movies['companies_names'] = df_movies['production_companies'].apply(extraer)

#extraer mas datos anidados columnas spoken_languages production_countries
df_movies['spoken_languages_clear'] = df_movies['spoken_languages'].apply(extraer2)
df_movies['production_countries_clean'] = df_movies['production_countries'].apply(extraer2)


In [6]:
#borro las columnas
df_movies.drop(columns=['production_companies','genres','production_countries','spoken_languages'], inplace=True)

In [7]:
# Convertir la columna 'belongs_to_collection' de cadenas a diccionarios
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


# Crear nuevas columnas a partir de los datos en la columna 'info' cambiar a collection
#df_movies['id_collection'] = df_movies['belongs_to_collection'].apply(lambda x: x.get('id') if isinstance(x, dict) else 0	)
df_movies['name_collection'] = df_movies['belongs_to_collection'].apply(lambda x: x.get('name') if isinstance(x, dict) else 0	)

#borrar columna belongs_to_collection parab quedarme con #NAME_COLLECTION
df_movies.drop(columns=['belongs_to_collection'], inplace=True)

In [8]:
#convertir a int o float columnas y los errores poner en nulo
df_movies['budget'] = pd.to_numeric(df_movies['budget'], errors='coerce')

df_movies['popularity'] = pd.to_numeric(df_movies['popularity'], errors='coerce')
df_movies['runtime'] = pd.to_numeric(df_movies['runtime'], errors='coerce')
df_movies['vote_average'] = pd.to_numeric(df_movies['vote_average'], errors='coerce')
df_movies['vote_count'] = pd.to_numeric(df_movies['vote_count'], errors='coerce')

In [9]:
#borrar columnas segun requerimiento
df_movies.drop(columns=['video', 'imdb_id','adult', 'original_title','poster_path','homepage'], inplace=True)

# Eliminar filas con valores nulos en 'release_date'
df_movies = df_movies.dropna(subset=['release_date'])


In [10]:
# Reemplazar valores nulos en 'revenue' por 0 y pasa a int
df_movies['revenue'] = df_movies['revenue'].fillna(0).astype(int)

In [11]:
#creo return y evito división por cero
df_movies['return'] = df_movies.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] != 0 else 0, axis=1)

In [12]:
#verificar duplicados
df_duplicados = df_movies.duplicated()
num_duplicados = df_duplicados.sum()
print(f"Número de filas duplicadas: {num_duplicados}")

Número de filas duplicadas: 17


In [13]:
# Seleccionar las primeras 17 filas duplicadas
duplicated_indices = df_duplicados[df_duplicados].index[:17]

df_movies = df_movies.drop(index=duplicated_indices)

In [14]:
#borro tagline ya que tiene muchos nulos y no considero que sea un campo relevante ya que almacena frases
df_movies.drop(columns='tagline', inplace=True)


In [15]:
df_movies.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,title,vote_average,vote_count,genres_clean,companies_names,spoken_languages_clear,production_countries_clean,name_collection,return
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81.0,Released,Toy Story,7.7,5415.0,"Animation, Comedy, Family",Pixar Animation Studios,English,United States of America,Toy Story Collection,12.451801
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249,104.0,Released,Jumanji,6.9,2413.0,"Adventure, Fantasy, Family","TriStar Pictures, Teitler Film, Interscope Com...","English, Français",United States of America,0,4.043035
2,0.0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0,101.0,Released,Grumpier Old Men,6.5,92.0,"Romance, Comedy","Warner Bros., Lancaster Gate",English,United States of America,Grumpy Old Men Collection,0.000000
3,16000000.0,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156,127.0,Released,Waiting to Exhale,6.1,34.0,"Comedy, Drama, Romance",Twentieth Century Fox Film Corporation,English,United States of America,0,5.090760
4,0.0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911,106.0,Released,Father of the Bride Part II,5.7,173.0,Comedy,"Sandollar Productions, Touchstone Pictures",English,United States of America,Father of the Bride Collection,0.000000


In [16]:
# Contar los valores nulos en cada columna
df_movies.isnull().sum()


budget                          3
id                              0
original_language              11
overview                      941
popularity                      3
release_date                    0
revenue                         0
runtime                       249
status                         83
title                           3
vote_average                    3
vote_count                      3
genres_clean                    0
companies_names                 0
spoken_languages_clear          0
production_countries_clean      0
name_collection                 0
return                          3
dtype: int64

In [17]:
#borrar nulos 
df_movies.dropna(inplace = True)

In [18]:
#hacer que release_date sea tipo datatime 
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'])

#crear columna release_year
df_movies['release_year'] = df_movies['release_date'].dt.year

In [19]:
#chequeo nulos
df_movies.isnull().sum()

budget                        0
id                            0
original_language             0
overview                      0
popularity                    0
release_date                  0
revenue                       0
runtime                       0
status                        0
title                         0
vote_average                  0
vote_count                    0
genres_clean                  0
companies_names               0
spoken_languages_clear        0
production_countries_clean    0
name_collection               0
return                        0
release_year                  0
dtype: int64

In [20]:
df_movies.shape

(44344, 19)

In [21]:
#asi queda mi dataframe
df_movies.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,title,vote_average,vote_count,genres_clean,companies_names,spoken_languages_clear,production_countries_clean,name_collection,return,release_year
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81.0,Released,Toy Story,7.7,5415.0,"Animation, Comedy, Family",Pixar Animation Studios,English,United States of America,Toy Story Collection,12.451801,1995
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249,104.0,Released,Jumanji,6.9,2413.0,"Adventure, Fantasy, Family","TriStar Pictures, Teitler Film, Interscope Com...","English, Français",United States of America,0,4.043035,1995
2,0.0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0,101.0,Released,Grumpier Old Men,6.5,92.0,"Romance, Comedy","Warner Bros., Lancaster Gate",English,United States of America,Grumpy Old Men Collection,0.000000,1995
3,16000000.0,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156,127.0,Released,Waiting to Exhale,6.1,34.0,"Comedy, Drama, Romance",Twentieth Century Fox Film Corporation,English,United States of America,0,5.090760,1995
4,0.0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911,106.0,Released,Father of the Bride Part II,5.7,173.0,Comedy,"Sandollar Productions, Touchstone Pictures",English,United States of America,Father of the Bride Collection,0.000000,1995


## COMIENZO CON EL DATASET CREDITS

In [24]:
df_credits = pd.read_csv(r'D:\Denise_Estudio\henry\PI\Data\credits.csv')

In [25]:
#funcion extrae datos anidados de cast
def extraer_cast(list):
    try:
        if list != []:
            list_aux= ast.literal_eval(list)

            return ', '.join([dic['name'] for dic in list_aux])
        else: 
            return 'no register'
        
    except:
        return 'no register'
    
# Definir la función extraer que filtra y devuelve el nombre del director
def extraer_director(crew_list):
    try:
        # Convertir la cadena de texto a una lista de diccionarios
        list_aux = ast.literal_eval(crew_list)
        
        # Buscar y devolver el nombre del director
        for dic in list_aux:
            if dic.get('job') == 'Director':
                return dic.get('name', 'no register')
        return 'no register'  # Si no se encuentra un director, devolver 'no register'
    except (ValueError, SyntaxError):
        # Manejar posibles errores
        return 'no register'

In [26]:
#desanido cast
df_credits['cast_clean'] =df_credits['cast'].apply(extraer_cast)
#desanido crew solo busco director
df_credits['director'] =df_credits['crew'].apply(extraer_director)



In [27]:
df_credits.head()

,cast,crew,id,cast_clean,director
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer


In [28]:
#borro las columnas innecesarias
df_credits.drop(columns=['crew','cast'], inplace=True)

In [29]:
df_credits.head()

,id,cast_clean,director
0,862,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,8844,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston
2,15602,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch
3,31357,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker
4,11862,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer


In [30]:
df_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          45476 non-null  int64 
 1   cast_clean  45476 non-null  object
 2   director    45476 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [31]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44344 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   budget                      44344 non-null  float64       
 1   id                          44344 non-null  object        
 2   original_language           44344 non-null  object        
 3   overview                    44344 non-null  object        
 4   popularity                  44344 non-null  float64       
 5   release_date                44344 non-null  datetime64[ns]
 6   revenue                     44344 non-null  int64         
 7   runtime                     44344 non-null  float64       
 8   status                      44344 non-null  object        
 9   title                       44344 non-null  object        
 10  vote_average                44344 non-null  float64       
 11  vote_count                  44344 non-null  float64       


In [32]:
#paso id a integer
df_movies['id'] = pd.to_numeric(df_movies['id'],  errors='coerce', downcast='integer')

In [33]:
#uno mis dos dataframes
data = pd.merge(df_movies, df_credits, on='id', how='inner')

In [34]:
data.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,title,...,vote_count,genres_clean,companies_names,spoken_languages_clear,production_countries_clean,name_collection,return,release_year,cast_clean,director
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81.0,Released,Toy Story,...,5415.0,"Animation, Comedy, Family",Pixar Animation Studios,English,United States of America,Toy Story Collection,12.451801,1995,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249,104.0,Released,Jumanji,...,2413.0,"Adventure, Fantasy, Family","TriStar Pictures, Teitler Film, Interscope Com...","English, Français",United States of America,0,4.043035,1995,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston
2,0.0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0,101.0,Released,Grumpier Old Men,...,92.0,"Romance, Comedy","Warner Bros., Lancaster Gate",English,United States of America,Grumpy Old Men Collection,0.000000,1995,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch
3,16000000.0,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156,127.0,Released,Waiting to Exhale,...,34.0,"Comedy, Drama, Romance",Twentieth Century Fox Film Corporation,English,United States of America,0,5.090760,1995,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker
4,0.0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911,106.0,Released,Father of the Bride Part II,...,173.0,Comedy,"Sandollar Productions, Touchstone Pictures",English,United States of America,Father of the Bride Collection,0.000000,1995,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer


In [35]:
data.shape

(44400, 21)

In [1]:
# Guardar el DataFrame optimizado en un archivo CSV en una ruta específica

ruta = r'D:\Denise_Estudio\henry\PI\Data\movies_data.csv'

In [38]:
data.to_csv(ruta, index=False)